In [108]:
import email
import boto3
# maybe pynamodb?
from pathlib import Path

mail_folder = "/home/rjanssen/Mail/new"
# mail = "1693230892.M909969P24382Q15Rd92b253b3507a6ec.ENG-RJANSSEN-LT" # musictick.com, 1 header and 2 payload parts
mail = "1693230881.M562431P24282Q3Rf99d4a372aa69efc.ENG-RJANSSEN-LT" # faz.net, only singlepart HTML, 

with open(Path(mail_folder, mail)) as ml:
    msg = email.message_from_file(ml)

In [77]:
msg.is_multipart()

False

In [ ]:
# this is about the header
msg.get_content_type()
#msg.get_content_maintype() 
#msg.get_content_subtype()

In [75]:
for part in msg.walk():
    print(part.get_content_type())

text/html


In [67]:
msg.get_charsets() # also tells you how many parts the email has

[None, 'us-ascii', 'us-ascii']

In [113]:
msg

In [78]:
msg.get_params()

[('text/html', ''), ('charset', 'utf-8')]

In [111]:
"M562431P24282Q3Rf99d4a372aa69efc" in msg

False

In [79]:
print(msg)

Return-Path: <
  bounce-35325_HTML-879604774-3086750-7331165-28907@bounce.nl.faz.net>
Delivered-To: unknown
Received: from outlook.office365.com (52.96.66.226:995) by
  ENG-RJANSSEN-LT.eu.ad.cuda-inc.com with POP3-SSL getmail6 msgid:14; 28 Aug
  2023 13:54:41 -0000
Received: from AS8PR01MB8698.eurprd01.prod.exchangelabs.com
 (2603:10a6:20b:44d::12) by VI1PR01MB4158.eurprd01.prod.exchangelabs.com with
 HTTPS; Fri, 25 Aug 2023 09:46:46 +0000
Received: from AS8PR07CA0005.eurprd07.prod.outlook.com (2603:10a6:20b:451::32)
 by AS8PR01MB8698.eurprd01.prod.exchangelabs.com (2603:10a6:20b:44d::12) with
 Microsoft SMTP Server (version=TLS1_2,
 cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6723.18; Fri, 25 Aug
 2023 09:46:45 +0000
Received: from VI1EUR03FT051.eop-EUR03.prod.protection.outlook.com
 (2603:10a6:20b:451:cafe::dc) by AS8PR07CA0005.outlook.office365.com
 (2603:10a6:20b:451::32) with Microsoft SMTP Server (version=TLS1_2,
 cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.

In [60]:

#dir(msg)

In [106]:
headers = msg._headers
type(headers)
#headers

list

In [105]:
payloads = msg.get_payload()
#print(f"payload 1: {payloads[0]}")
#print(f"payload 2: {payloads[1]}")
#payloads

In [59]:
#for item in msg:
   # print(item)

In [87]:
msg.get_unixfrom()

In [88]:
# Return the value of the named header field, same as get(name, failobj=None), but you can change the failobj
msg.__getitem__("date")

'Fri, 25 Aug 2023 03:46:43 -0600'

In [89]:
#print(msg.get("datex"))
#print(msg.__getitem__("datex"))

In [90]:
msg.__getitem__("subject")

'=?UTF-8?Q?Newsletterbest=C3=A4tigung_F.A.Z._Newsletter_Finanzen?='

In [91]:
msg.__getitem__("from")

'"Frankfurter Allgemeine Zeitung" <newsletter@nl.faz.net>'

In [92]:
# there are multiple "received" fields
msg.get_all("received")

['from outlook.office365.com (52.96.66.226:995) by\n  ENG-RJANSSEN-LT.eu.ad.cuda-inc.com with POP3-SSL getmail6 msgid:14; 28 Aug\n  2023 13:54:41 -0000',
 'from AS8PR01MB8698.eurprd01.prod.exchangelabs.com\n (2603:10a6:20b:44d::12) by VI1PR01MB4158.eurprd01.prod.exchangelabs.com with\n HTTPS; Fri, 25 Aug 2023 09:46:46 +0000',
 'from AS8PR07CA0005.eurprd07.prod.outlook.com (2603:10a6:20b:451::32)\n by AS8PR01MB8698.eurprd01.prod.exchangelabs.com (2603:10a6:20b:44d::12) with\n Microsoft SMTP Server (version=TLS1_2,\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6723.18; Fri, 25 Aug\n 2023 09:46:45 +0000',
 'from VI1EUR03FT051.eop-EUR03.prod.protection.outlook.com\n (2603:10a6:20b:451:cafe::dc) by AS8PR07CA0005.outlook.office365.com\n (2603:10a6:20b:451::32) with Microsoft SMTP Server (version=TLS1_2,\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6699.14 via Frontend\n Transport; Fri, 25 Aug 2023 09:46:45 +0000',
 'from mta.nl.faz.net (136.147.180.43) by\n VI1EUR03FT0

In [93]:
msg.get_all("content-type") # this might only be the content type within the header, the other two are in the payload

['text/html;\n\tcharset="utf-8"']

In [94]:
msg.__getitem__("return-path")

'<\n  bounce-35325_HTML-879604774-3086750-7331165-28907@bounce.nl.faz.net>'

In [96]:
msg.__getitem__("delivered-to")

'unknown'

In [99]:
#msg.get_payload()

In [102]:
dynamodb = boto3.resource("dynamodb", endpoint_url="http://localhost:8000", region_name="eu-central-1")

In [103]:
# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='emails',
    KeySchema=[
        {
            'AttributeName': 'mail_id',
            'KeyType': 'HASH' # partition key
        },
        {
            'AttributeName': 'mail_count',
            'KeyType': 'RANGE' # sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'mail_id',
            'AttributeType': 'S' # string
        },
        {
            'AttributeName': 'mail_count',
            'AttributeType': 'N'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5, # ReadCapacityUnits set to 5 strongly consistent reads per second
        'WriteCapacityUnits': 5 # WriteCapacityUnits set to 5 writes per second
    }
)

# Wait until the table exists.
table.wait_until_exists()
print("Status:", table.table_status)
# Print out some data about the table.
print(table.item_count)

Status: ACTIVE
0


In [104]:
new_mail_dict = {"mail_id":,
                 "mail_count": 1,
                 "content": {
                     "headers": msg._headers,
                     "payloads": msg.get_payloads(),
                 }
                 
                 
                 

[('Return-Path',
  '<\n  bounce-35325_HTML-879604774-3086750-7331165-28907@bounce.nl.faz.net>'),
 ('Delivered-To', 'unknown'),
 ('Received',
  'from outlook.office365.com (52.96.66.226:995) by\n  ENG-RJANSSEN-LT.eu.ad.cuda-inc.com with POP3-SSL getmail6 msgid:14; 28 Aug\n  2023 13:54:41 -0000'),
 ('Received',
  'from AS8PR01MB8698.eurprd01.prod.exchangelabs.com\n (2603:10a6:20b:44d::12) by VI1PR01MB4158.eurprd01.prod.exchangelabs.com with\n HTTPS; Fri, 25 Aug 2023 09:46:46 +0000'),
 ('Received',
  'from AS8PR07CA0005.eurprd07.prod.outlook.com (2603:10a6:20b:451::32)\n by AS8PR01MB8698.eurprd01.prod.exchangelabs.com (2603:10a6:20b:44d::12) with\n Microsoft SMTP Server (version=TLS1_2,\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.6723.18; Fri, 25 Aug\n 2023 09:46:45 +0000'),
 ('Received',
  'from VI1EUR03FT051.eop-EUR03.prod.protection.outlook.com\n (2603:10a6:20b:451:cafe::dc) by AS8PR07CA0005.outlook.office365.com\n (2603:10a6:20b:451::32) with Microsoft SMTP Server (versio